# Advanced cis-QTL analysis with individual level data

This notebook performs various advanced statistical analysis on multiple xQTL in a given region. Current procedures implemented include:

1. Univariate analysis
    - SuSiE
    - Univeriate TWAS weights: LASSO, Elastic net, mr.mash and SuSiE (optional)
    - Cross validation of TWAS methods (optional but highly recommended if TWAS weights are computed)
2. Functional data (epigenomic xQTL) analysis
    - fSuSiE
3. Multivariate analysis
    - mvSuSiE
    - mr.mash

## Input

1. A list of regions to be analyzed (optional); the last column of this file should be region name.
2. Either a list of per chromosome genotype files, or one file for genotype data of the entire genome. Genotype data has to be in PLINK `bed` format. 
3. Vector of lists of phenotype files per region to be analyzed, in UCSC `bed.gz` with index in `bed.gz.tbi` formats.
4. Vector of covariate files corresponding to the lists above.
5. Customized cis windows file. If it is not provided, a fixed sized cis-window will be used.
6. Optionally a vector of names of the phenotypic conditions in the form of `cond1 cond2 cond3` separated with whitespace.

Input 2 and 3 should be outputs from `genotype_per_region` and `annotate_coord` modules in previous preprocessing steps. 4 should be output of `covariate_preprocessing` pipeline that contains genotype PC, phenotypic hidden confounders and fixed covariates.

### Example genotype data

```
#chr        path
chr21 /mnt/mfs/statgen/xqtl_workflow_testing/protocol_example.genotype.chr21.bed
chr22 /mnt/mfs/statgen/xqtl_workflow_testing/protocol_example.genotype.chr22.bed
```

Alternatively, simply use `protocol_example.genotype.chr21_22.bed` if all chromosomes are in the same file.

### Example phenotype list

```
#chr    start   end ID  path
chr12   752578  752579  ENSG00000060237  /home/gw/GIT/github/fungen-xqtl-analysis/analysis/Wang_Columbia/ROSMAP/MWE/output/phenotype/protocol_example.protein.bed.gz
chr12   990508  990509  ENSG00000082805  /home/gw/GIT/github/fungen-xqtl-analysis/analysis/Wang_Columbia/ROSMAP/MWE/output/phenotype/protocol_example.protein.bed.gz
chr12   2794969 2794970 ENSG00000004478  /home/gw/GIT/github/fungen-xqtl-analysis/analysis/Wang_Columbia/ROSMAP/MWE/output/phenotype/protocol_example.protein.bed.gz
chr12   4649113 4649114 ENSG00000139180  /home/gw/GIT/github/fungen-xqtl-analysis/analysis/Wang_Columbia/ROSMAP/MWE/output/phenotype/protocol_example.protein.bed.gz
chr12   6124769 6124770 ENSG00000110799  /home/gw/GIT/github/fungen-xqtl-analysis/analysis/Wang_Columbia/ROSMAP/MWE/output/phenotype/protocol_example.protein.bed.gz
chr12   6534516 6534517 ENSG00000111640  /home/gw/GIT/github/fungen-xqtl-analysis/analysis/Wang_Columbia/ROSMAP/MWE/output/phenotype/protocol_example.protein.bed.gz
```

### Example cis-window file

It should have strictly 4 columns, with the header a commented out line:

```
#chr	start	end	gene_id
chr1	0	6480000	ENSG00000008128
chr1	0	6480000	ENSG00000008130
chr1	0	6480000	ENSG00000067606
chr1	0	7101193	ENSG00000069424
chr1	0	7960000	ENSG00000069812
chr1	0	6480000	ENSG00000078369
chr1	0	6480000	ENSG00000078808
```

The key is that the 4th column ID should match with the 4th column ID in the phenotype list. Otherwise the cis-window to analyze will not be found.

### About indels

Option `--no-indel` will remove indel from analysis. FIXME: Gao need to provide more guidelines how to deal with indels in practice.

## Output

For each analysis region, the output is SuSiE model fitted and saved in RDS format.

## Minimal working example

### SuSiE with TWAS weights

Below we duplicate the examples for phenotype and covariates to demonstrate that when there are multiple phenotypes for the same genotype it is possible to use this pipeline to analyze all of them (more than two is accepted as well)

```
# suggested output naming convention is cohort_modality, eg ROSMAP_snRNA_pseudobulk
sos run xqtl-pipeline/pipeline/cis_workhorse.ipynb susie_twas \
    --name protocol_example_protein \
    --genoFile xqtl_association/protocol_example.genotype.chr21_22.bed \
    --phenoFile output/phenotype/protocol_example.protein.region_list.txt \
                output/phenotype/protocol_example.protein.region_list.txt \
    --covFile output/covariate/protocol_example.protein.protocol_example.samples.protocol_example.genotype.chr21_22.pQTL.plink_qc.prune.pca.Marchenko_PC.gz \
              output/covariate/protocol_example.protein.protocol_example.samples.protocol_example.genotype.chr21_22.pQTL.plink_qc.prune.pca.Marchenko_PC.gz \
    --customized-cis-windows xqtl_association/protocol_example.protein.enhanced_cis_chr21_chr22.bed \
    --region-name ENSG00000241973_P42356 ENSG00000160209_O00764 ENSG00000100412_Q99798 \
    --phenotype-names trait_A trait_B \
    --container oras://ghcr.io/cumc/pecotmr_apptainer:latest
```

It is also possible to only analyze a selected list of regions by name, using either option `--region-list` or option `--region-name` or both. The command below will generate 6 regions to analyze:

In [2]:
import pandas as pd

# Define the data
data = {
    "#chr": ["chr1", "chr1", "chr1"],
    "start": [1000000, 1010000, 1020000],
    "end": [1000100, 1010100, 1020100],
    "ID": ["ENSG00000159082_O43426", "ENSG00000159131_P22102", "ENSG00000205726_Q15811"]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a tab-separated file
df.to_csv("output/selected_regions.tsv", sep="\t", index=False)

```
sos run pipeline/cis_workhorse.ipynb susie_twas \
        --name protocol_example_protein \
        --genoFile output/protocol_example.protein.enhanced_cis_chr21_chr22_genotype_by_region/protocol_example.genotype.chr21_22.genotype_by_region_files.txt \
        --phenoFile output/phenotype/protocol_example.protein.region_list.txt \
                    output/phenotype/protocol_example.protein.region_list.txt \
        --covFile output/covariate/protocol_example.protein.protocol_example.samples.protocol_example.genotype.chr21_22.pQTL.plink_qc.prune.pca.Marchenko_PC.gz \
                  output/covariate/protocol_example.protein.protocol_example.samples.protocol_example.genotype.chr21_22.pQTL.plink_qc.prune.pca.Marchenko_PC.gz \
       --phenotype-names A B \
        --cwd output/ \
        --region-list output/selected_regions.tsv \
        --region-name ENSG00000154654_O15394 ENSG00000142192_P05067 ENSG00000159082_O43426 \
        --no-indel \
        --container oras://ghcr.io/cumc/pecotmr_apptainer:latest
```

To exclude cross-validation in TWAS weights computation,

```
sos run pipeline/cis_workhorse.ipynb susie_twas --no-twas-cv \
        --name protocol_example_protein \
        --genoFile output/protocol_example.protein.enhanced_cis_chr21_chr22_genotype_by_region/protocol_example.genotype.chr21_22.genotype_by_region_files.txt \
        --phenoFile output/phenotype/protocol_example.protein.region_list.txt \
                    output/phenotype/protocol_example.protein.region_list.txt \
        --covFile output/covariate/protocol_example.protein.protocol_example.samples.protocol_example.genotype.chr21_22.pQTL.plink_qc.prune.pca.Marchenko_PC.gz \
                  output/covariate/protocol_example.protein.protocol_example.samples.protocol_example.genotype.chr21_22.pQTL.plink_qc.prune.pca.Marchenko_PC.gz \
       --phenotype-names A B \
        --cwd output/ \
        --region-list output/selected_regions.tsv \
        --region-name ENSG00000154654_O15394 ENSG00000142192_P05067 ENSG00000159082_O43426 \
        --no-indel \
            --container oras://ghcr.io/cumc/pecotmr_apptainer:latest
```

To perform fine-mapping only without TWAS weights,

```
sos run pipeline/cis_workhorse.ipynb susie_twas --no-twas-weights \
        --name protocol_example_protein \
        --genoFile output/protocol_example.protein.enhanced_cis_chr21_chr22_genotype_by_region/protocol_example.genotype.chr21_22.genotype_by_region_files.txt \
        --phenoFile output/phenotype/protocol_example.protein.region_list.txt \
                    output/phenotype/protocol_example.protein.region_list.txt \
        --covFile output/covariate/protocol_example.protein.protocol_example.samples.protocol_example.genotype.chr21_22.pQTL.plink_qc.prune.pca.Marchenko_PC.gz \
                  output/covariate/protocol_example.protein.protocol_example.samples.protocol_example.genotype.chr21_22.pQTL.plink_qc.prune.pca.Marchenko_PC.gz \
       --phenotype-names A B \
        --cwd output/ \
        --region-list output/selected_regions.tsv \
        --region-name ENSG00000154654_O15394 ENSG00000142192_P05067 ENSG00000159082_O43426 \
        --no-indel \
            --container oras://ghcr.io/cumc/pecotmr_apptainer:latest
```

It is also possible to specify a subset of samples to analyze, using `--keep-samples` parameter. For example we create a file to keep the ID of 50 samples,

In [ ]:
zcat output/covariate/protocol_example.protein.protocol_example.samples.protocol_example.genotype.chr21_22.pQTL.plink_qc.prune.pca.Marchenko_PC.gz | head -1 | awk '{for (i=2; i<=51; i++) printf $i " "; print ""}'> output/keep_samples.txt

then use them in our analysis,

In [ ]:
sos run xqtl-pipeline/pipeline/cis_workhorse.ipynb susie_twas \
    --name protocol_example_protein \
    --genoFile output/protocol_example.protein.enhanced_cis_chr21_chr22_genotype_by_region/protocol_example.genotype.chr21_22.genotype_by_region_files.txt \
    --phenoFile output/phenotype/protocol_example.protein.region_list.txt \
                output/phenotype/protocol_example.protein.region_list.txt \
    --covFile output/covariate/protocol_example.protein.protocol_example.samples.protocol_example.genotype.chr21_22.pQTL.plink_qc.prune.pca.Marchenko_PC.gz \
              output/covariate/protocol_example.protein.protocol_example.samples.protocol_example.genotype.chr21_22.pQTL.plink_qc.prune.pca.Marchenko_PC.gz \
    --phenotype-names trait_A trait_B \
    --region-name ENSG00000154654_O15394 ENSG00000142192_P05067 ENSG00000159082_O43426 \
    --keep-samples output/keep_samples.txt \
    --container oras://ghcr.io/cumc/pecotmr_apptainer:latest  

### fSuSiE

```
# suggested output naming convention is cohort_modality, eg ROSMAP_snRNA_pseudobulk
sos run pipeline/cis_workhorse.ipynb fsusie \
    --name protocol_example_methylation \
    --genoFile output/phenotype_by_region/protocol_example.methylation.enhanced_cis_chr21_chr22_genotype_by_region/protocol_example.genotype.chr21_22.genotype_by_region_files.txt \
    --phenoFile output/phenotype_by_region/protocol_example.methylation.bed.phenotype_by_region_files.txt \
                output/phenotype_by_region/protocol_example.methylation.bed.phenotype_by_region_files.txt  \
    --covFile output/covariate/protocol_example.methylation.protocol_example.samples.protocol_example.genotype.chr21_22.pQTL.plink_qc.prune.pca.Marchenko_PC.gz \
              output/covariate/protocol_example.methylation.protocol_example.samples.protocol_example.genotype.chr21_22.pQTL.plink_qc.prune.pca.Marchenko_PC.gz \
    --container oras://ghcr.io/cumc/pecotmr_apptainer:latest
```

In [1]:
[global]
parameter: cwd = path("output")
# A list of file paths for genotype data, or the genotype data itself. 
parameter: genoFile = path
# One or multiple lists of file paths for phenotype data.
parameter: phenoFile = paths
# Covariate file path
parameter: covFile = paths
# Optional: if a region list is provide the analysis will be focused on provided region. 
# The LAST column of this list will contain the ID of regions to focus on
# Otherwise, all regions with both genotype and phenotype files will be analyzed
parameter: region_list = path()
# Optional: if a region name is provided 
# the analysis would be focused on the union of provides region list and region names
parameter: region_name = []
# Only focus on a subset of samples
parameter: keep_samples = path()
# An optional list documenting the custom cis window for each region to analyze, with four column, chr, start, end, region ID (eg gene ID).
# If this list is not provided, the default `window` parameter (see below) will be used.
parameter: customized_cis_windows = path()
# Specify the cis window for the up and downstream radius to analyze around the region of interest in units of bp
# When this is zero, we will rely on customized_cis_windows
parameter: window = 0
# It is required to input the name of the analysis
parameter: name = str
parameter: container = ""
import re
parameter: entrypoint= ('micromamba run -a "" -n' + ' ' + re.sub(r'(_apptainer:latest|_docker:latest|\.sif)$', '', container.split('/')[-1])) if container else ""
# For cluster jobs, number commands to run per job
parameter: job_size = 200
# Wall clock time expected
parameter: walltime = "100h"
# Memory expected
parameter: mem = "80G"
# Number of threads
parameter: numThreads = 1
# Name of phenotypes
parameter: phenotype_names = [f'{x:bn}' for x in phenoFile]
parameter: seed = 999

def group_by_region(lst, partition):
    # from itertools import accumulate
    # partition = [len(x) for x in partition]
    # Compute the cumulative sums once
    # cumsum_vector = list(accumulate(partition))
    # Use slicing based on the cumulative sums
    # return [lst[(cumsum_vector[i-1] if i > 0 else 0):cumsum_vector[i]] for i in range(len(partition))]
    return partition

import os
import pandas as pd

def adapt_file_path(file_path, reference_file):
    """
    Adapt a single file path based on its existence and a reference file's path.

    Args:
    - file_path (str): The file path to adapt.
    - reference_file (str): File path to use as a reference for adaptation.

    Returns:
    - str: Adapted file path.

    Raises:
    - FileNotFoundError: If no valid file path is found.
    """
    reference_path = os.path.dirname(reference_file)

    # Check if the file exists
    if os.path.isfile(file_path):
        return file_path

    # Check file name without path
    file_name = os.path.basename(file_path)
    if os.path.isfile(file_name):
        return file_name

    # Check file name in reference file's directory
    file_in_ref_dir = os.path.join(reference_path, file_name)
    if os.path.isfile(file_in_ref_dir):
        return file_in_ref_dir

    # Check original file path prefixed with reference file's directory
    file_prefixed = os.path.join(reference_path, file_path)
    if os.path.isfile(file_prefixed):
        return file_prefixed

    # If all checks fail, raise an error
    raise FileNotFoundError(f"No valid path found for file: {file_path}")

def adapt_file_path_all(df, column_name, reference_file):
    return df[column_name].apply(lambda x: adapt_file_path(x, reference_file))

In [1]:
[get_analysis_regions: shared = "regional_data"]
# input is genoFile, phenoFile, covFile and optionally region_list. If region_list presents then we only analyze what's contained in the list.
# regional_data should be a dictionary like:
#{'data': [("genotype_1.bed", "phenotype_1.bed.gz", "covariate_1.gz"), ("genotype_2.bed", "phenotype_1.bed.gz", "phenotype_2.bed.gz", "covariate_1.gz", "covariate_2.gz") ... ],
# 'meta_info': [("chr12:752578-752579", "gene_1", "trait_1"), ("chr13:852580-852581", "gene_2", "trait_1", "trait_2") ... ]}

def process_pheno_files(pheno_files, cov_files, phenotype_names):
    '''
    Example output:
    #chr    start      end    start_cis       end_cis           ID  path     cov_path             cond             coordinate     geno_path
    0  chr12   752578   752579  652578   852579  ENSG00000060237_Q9H4A3  protocol_example.protein_1.bed.gz,protocol_example.protein_2.bed.gz  covar_1.gz,covar_2.gz  trait_A,trait_B    chr12:752578-752579  protocol_example.genotype.chr21_22.bed
    '''
    # Initialize an empty DataFrame for accumulation
    accumulated_pheno_df = pd.DataFrame()

    merge_keys = ['#chr', 'start', 'end', 'ID']

    for pheno_path, cov_path, phenotype_name in zip(pheno_files, cov_files, phenotype_names):
        if not os.path.isfile(cov_path):
            raise FileNotFoundError(f"No valid path found for file: {cov_path}")

        # Read and process each phenotype file
        pheno_df = pd.read_csv(pheno_path, sep="\t", header=0)
        
        # Adapt pheno file paths and add additional information
        pheno_df.iloc[:, 4] = adapt_file_path_all(pheno_df, pheno_df.columns[4], f"{pheno_path:a}")
        pheno_df = pheno_df.assign(
            cov_path=cov_path, 
            cond=phenotype_name)

        # Merge with the accumulated DataFrame
        if accumulated_pheno_df.empty:
            accumulated_pheno_df = pheno_df
        else:
            # Merge on specified keys with default suffixes
            merged_df = pd.merge(accumulated_pheno_df, pheno_df, on=merge_keys, how='outer', suffixes=('_x', '_y'))

            # Determine non-key columns
            non_key_columns = [col for col in pheno_df.columns if col not in merge_keys]

            # Concatenate non-key columns for matching keys
            for col in non_key_columns:
                col_x = f'{col}_x'
                col_y = f'{col}_y'

                # Handling concatenation for matching keys
                merged_df[col] = merged_df.apply(
                    lambda row: row[col_x] if pd.isna(row[col_y]) else 
                                (row[col_y] if pd.isna(row[col_x]) else f'{row[col_x]},{row[col_y]}'), axis=1)

                # Drop the temporary columns
                merged_df.drop([col_x, col_y], axis=1, inplace=True)

            accumulated_pheno_df = merged_df
    return accumulated_pheno_df

# Load phenotype meta data
if len(phenoFile) != len(covFile):
    raise ValueError("Number of input phenotypes files must match that of covariates files")
if len(phenoFile) != len(phenotype_names):
    raise ValueError("Number of input phenotypes files must match the number of phenotype names")
meta_data = process_pheno_files(phenoFile, covFile, phenotype_names)

# Load genotype meta data
if f"{genoFile:x}" == ".bed":
    geno_meta_data = pd.DataFrame([("chr"+str(x), f"{genoFile:a}") for x in range(1,23)] + [("chrX", f"{genoFile:a}")], columns=['#chr', 'geno_path'])
else:
    geno_meta_data = pd.read_csv(f"{genoFile:a}", sep = "\t", header=0)
    geno_meta_data.iloc[:, 1] = adapt_file_path_all(geno_meta_data, geno_meta_data.columns[1], f"{genoFile:a}")
    geno_meta_data.columns = ['#chr', 'geno_path']
    geno_meta_data['#chr'] = geno_meta_data['#chr'].apply(lambda x: x if x.startswith('chr') else f'chr{x}')

# Checking the DataFrame
valid_chr_values = [f'chr{x}' for x in range(1, 23)] + ['chrX']
if not all(value in valid_chr_values for value in geno_meta_data['#chr']):
    raise ValueError("Invalid chromosome values found. Allowed values are chr1 to chr22 and chrX.")

meta_data = meta_data.merge(geno_meta_data, on='#chr', how='inner')

if len(meta_data.index) == 0:
    raise ValueError("No region overlap between genotype and any of the phenotypes")

region_ids = []
# If region_list is provided, read the file and extract IDs
if region_list.is_file():
    region_list_df = pd.read_csv(region_list, sep = "\t", header=None, comment = "#")
    region_ids = region_list_df.iloc[:, -1].unique()  # Extracting the last column for IDs

# If region_name is provided, include those IDs as well
# --region-name A B C will result in a list of ["A", "B", "C"] here
if len(region_name) > 0:
    region_ids = list(set(region_ids).union(set(region_name)))

# If either region_list or region_name is provided, filter the meta_data
if len(region_ids) > 0:
    meta_data = meta_data[meta_data['ID'].isin(region_ids)]

# Adjust cis-window
if os.path.isfile(customized_cis_windows):
    print(f"Loading customized cis-window data from {customized_cis_windows}")
    cis_list = pd.read_csv(customized_cis_windows, comment="#", header=None, names=["#chr","start","end","ID"], sep="\t")
    meta_data = pd.merge(meta_data, cis_list, on=['#chr', 'ID'], how='left', suffixes=('', '_cis')) 
    mismatches = meta_data[meta_data['start_cis'].isna()]
    if not mismatches.empty:
        print("First 5 mismatches:")
        print(mismatches[['ID']].head())
        raise ValueError(f"{len(mismatches)} regions to analyze cannot be found in ``{customized_cis_windows}``. Please check your ``{customized_cis_windows}`` database to make sure it contains all cis-window definitions. ")
else:
    if window <=0 :
        raise ValueError("Please either input cis-window file ``--customized-cis-windows`` or set ``--window`` to a positive integer")
    meta_data['start_cis'] = meta_data['start'].apply(lambda x: max(x - window, 0))
    meta_data['end_cis'] = meta_data['end'] + window
# Create the final dictionary
regional_data = {
    'data': [(row['geno_path'], *row['path'].split(','), *row['cov_path'].split(',')) for _, row in meta_data.iterrows()],
    'meta_info': [(f"{row['#chr']}:{row['start']}-{row['end']}", # this is the phenotype region
                   f"{row['#chr']}:{row['start_cis']}-{row['end_cis']}", # this is the cis-window region
                   row['ID'], *row['cond'].split(',')) for _, row in meta_data.iterrows()]
}

## Univariate analysis: SuSiE and TWAS

In [1]:
[susie_twas_1]
# initial number of single effects for SuSiE
parameter: init_L = 8
# maximum number of single effects to use for SuSiE
parameter: max_L = 30
# remove a variant if it has more than imiss missing individual level data
parameter: imiss = 1.0
# MAF cutoff
parameter: maf = 0.005
# MAC cutoff, on top of MAF cutoff
parameter: mac = 5
# Remove indels if indel = False
parameter: indel = True
parameter: pip_cutoff = 0.1
parameter: coverage = [0.95, 0.7, 0.5]
# Compute TWAS weights as well
parameter: twas_weights = True
# Perform K folds valiation CV for TWAS
# Set it to zero if this is to be skipped
parameter: twas_cv_folds = 5
depends: sos_variable("regional_data")

meta_info = regional_data['meta_info']
input: regional_data["data"], group_by = lambda x: group_by_region(x, regional_data["data"]), group_with = "meta_info"
output: f'{cwd:a}/{step_name[:-2]}/{name}.{_meta_info[2]}.susie{"_weights_db" if twas_weights else ""}.rds'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stdout = f"{_output:n}.stdout", stderr = f"{_output:n}.stderr", container = container, entrypoint = entrypoint
    library(pecotmr)
    # extract subset of samples
    keep_samples = NULL
    if (${"TRUE" if keep_samples.is_file() else "FALSE"}) {
      keep_samples = unlist(strsplit(readLines(${keep_samples:ar}), "\\s+"))
      message(paste(length(keep_samples), "samples are selected to be loaded for analysis"))
    }
    # Load regional association data
    tryCatch({
    fdat = load_regional_univariate_data(genotype = ${_input[0]:anr},
                                          phenotype = c(${",".join(['"%s"' % x.absolute() for x in _input[1:len(_input)//2+1]])}),
                                          covariate = c(${",".join(['"%s"' % x.absolute() for x in _input[len(_input)//2+1:]])}),
                                          region = "${_meta_info[0]}",
                                          cis_window = "${_meta_info[1]}",
                                          conditions = c(${",".join(['"%s"' % x for x in _meta_info[3:]])}),
                                          maf_cutoff = ${maf},
                                          mac_cutoff = ${mac},
                                          imiss_cutoff = ${imiss},
                                          keep_indel = ${"TRUE" if indel else "FALSE"},
                                          keep_samples = keep_samples,
                                          scale_residuals = FALSE)
    }, NoSNPsError = function(e) {
        message("Error: ", e$message)
        #saveRDS(NULL, ${_output:ar})
        saveRDS(list("${_meta_info[2] + '@' + _meta_info[1]}" = e$message), ${_output:ar}, compress='xz')
        quit(save="no")
    })
    # Univeriate analysis suite
    fitted = list()
    for (r in 1:length(fdat$residual_Y)) {
      st = proc.time()
      fitted[[r]] = susie_wrapper(fdat$residual_X[[r]], fdat$residual_Y[[r]], ${init_L}, ${max_L}, ${coverage[0]})
      fitted[[r]] = susie_post_processor(fitted[[r]], fdat$residual_X[[r]], fdat$residual_Y[[r]], fdat$residual_X_scalar[[r]], fdat$residual_Y_scalar[[r]], 
                                       fdat$maf[[r]], secondary_coverage = c(${",".join([str(x) for x in coverage[1:]])}), signal_cutoff = ${pip_cutoff}, 
                                       other_quantities = list(dropped_samples = fdat$dropped_sample[[r]]))
      if ( ${"TRUE" if twas_weights else "FALSE"} ) {
          weight_methods = list(enet_weights = list(),
                          lasso_weights = list(),
                          mrash_weights = list(init_prior_sd = TRUE)
                        )
          # if previously SuSiE fit has no potential signal at all, we will not fit SuSiE for TWAS here. Otherwise using precomputed model so we don't have to run it again
          if (length(fitted[[r]]$susie_result_trimmed) != 0) weight_methods$susie_weights = list(susie_fit = fitted[[r]]$susie_result_trimmed)
          fitted[[r]]$twas_weights = twas_weights(fdat$residual_X[[r]], fdat$residual_Y[[r]], weight_methods = weight_methods)
          if ( ${"TRUE" if twas_cv_folds > 0 else "FALSE"} ) {
              # Reset previously used SuSiE model from complete data in order to rerun SuSiE each fold
              if (length(fitted[[r]]$susie_result_trimmed) != 0) weight_methods$susie_weights = list()
              fitted[[r]]$twas_cv_result = twas_weights_cv(fdat$residual_X[[r]], fdat$residual_Y[[r]], fold = ${twas_cv_folds}, weight_methods = weight_methods, seed = ${seed})
          }
      }
    }
    fitted[[r]]$total_time_elapsed = proc.time() - st
    names(fitted) <- names(fdat$residual_Y)
    saveRDS(list("${_meta_info[2] + '@' + _meta_info[1]}" = fitted), ${_output:ar}, compress='xz')

## Multivariate analysis: mvSuSiE and mr.mash

### mvSuSiE

In [ ]:
[mvsusie_1]
# Prior model file generated from mashr. 
# Default will be used if it does not exist.
parameter: mixture_prior = path()
parameter: max_L = 20
# remove a variant if it has more than imiss missing individual level data
parameter: imiss = 0.1
# MAF cutoff
parameter: maf = 0.0
# MAC cutoff, on top of MAF cutoff
# Here I set default to mac = 10 rather than using an MAF cutoff
# I don't set it to 5 because I'm not so sure of performance of SuSiE on somewhat infrequent variants
# MAC = 10 would not be too infrequenty for xQTL data where sample size is about ~1,000 at most (as of 2022)
parameter: mac = 10
# Remove indels if indel = False
parameter: indel = True
depends: sos_variable("regional_data")

meta_info = regional_data['meta_info']
input: regional_data["data"], group_by = lambda x: group_by_region(x, regional_data["data"]), group_with = "meta_info"
output: f'{cwd:a}/{step_name[:-2]}/{name}.{_meta_info[0]}.susie_fitted.rds'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stdout = f"{_output:n}.stdout", stderr = f"{_output:n}.stderr", container = container, entrypoint = entrypoint
   
    get_prior_indices <- function(Y, U) {
      # make sure the prior col/rows match the colnames of the Y matrix
      y_names = colnames(Y)
      u_names = colnames(U)
      if (is.null(y_names) || is.null(u_names)) {
          return(NULL)
      } else if (identical(y_names, u_names)) {
          return(NULL)
      } else {
          return(match(y_names, u_names))
      }
    }

    # Load regional association data
    fdat = load_regional_multivariate_data(genotype = ${_input[0]:anr},
                                          phenotype = c(${",".join(['"%s"' % x.absolute() for x in _input[1::2]])}),
                                          covariate = c(${",".join(['"%s"' % x.absolute() for x in _input[2::2]])}),
                                          region = ${'"%s:%s-%s"' % (_meta_info[1], _meta_info[2], _meta_info[3])},
                                          maf_cutoff = ${maf},
                                          mac_cutoff = ${mac},
                                          imiss_cutoff = ${imiss},
                                          keep_indel = ${"TRUE" if indel else "FALSE"})

    # univariate summary statistics
    non_missing = lapply(1:ncol(fdat$residual_Y), function(r)) which(!is.na(fdat$residual_Y[,r]))
    univariate_res = lapply(1:ncol(fdat$residual_Y), function(r) susieR:::univariate_regression(X[non_missing[[r]], ], fdat$residual_Y[non_missing[[r]], r]))
    sumstat = list(bhat=do.call(cbind, lapply(1:ncol(fdat$residual_Y), function(r) univariate_res[[r]]$betahat)),
                   sbhat=do.call(cbind, lapply(1:ncol(fdat$residual_Y), function(r) univariate_res[[r]]$sebetahat)))
  
    # Multivariate fine-mapping
    # FIXME: handle it when prior does not exist
    prior = readRDS(${mixture_prior:r})
    print(paste("Number of components in the mixture prior:", length(prior$U)))
    prior = mvsusieR::create_mash_prior(mixture_prior=list(weights=prior$w, matrices=prior$U), include_indices = get_prior_indices(fdat$residual_Y, prior$U[[1]]), max_mixture_len=-1)   
    resid_Y = compute_cov_flash(fdat$residual_Y)
    st = proc.time()
    fitted = mvsusieR::mvsusie(fdat$X, 
                               fdat$residual_Y, 
                               L=${max_L}, 
                               prior_variance=prior, 
                               residual_variance=resid_Y, 
                               precompute_covariances=F, 
                               compute_objective=T, 
                               estimate_residual_variance=F, 
                               estimate_prior_variance=T, 
                               estimate_prior_method='EM',
                               max_iter = 200, 
                               n_thread=${numThreads}, 
                               approximate=F)
    fitted$analysis_time = proc.time() - st
    fitted$cs_corr = susieR::get_cs_correlation(fitted, X=fdat$X)
    fitted$cs_snps = names(fitted$X_column_scale_factors[unlist(fitted$sets$cs)])
    fitted$variable_name = names(fitted$pip)
    fitted$analysis_script = load_script()
    fitted$dropped_samples = fdat$dropped_sample
    fitted$sample_names = colnames(fdat$residual_Y)
    fitted$residual_y = resid_Y
    saveRDS(fitted, ${_output:ar})

### mr.mash

In [ ]:
[mrmash_1]
# Prior model file generated from mashr. 
# Default will be used if it does not exist.
parameter: mixture_prior = path()
# remove a variant if it has more than imiss missing individual level data
parameter: imiss = 0.1
# MAF cutoff
parameter: maf = 0.05
# MAC cutoff, on top of MAF cutoff
parameter: mac = 0
# Remove indels if indel = False
parameter: indel = True
# Path to prior grid data file: an RDS file with scaling factors
parameter: prior_grid = path('.')
# Path to prior weights data file: an RDS file with prior weights
parameter: prior_weights = path('.')
# Path to summary statistics directory
parameter: sumstats_file = path('.')
# Path to sample partition for cross validation
parameter: sample_partition = path('.')
parameter: fold = 1
parameter: var_cutoff = 0.05
parameter: n_nonmiss_Y = 100
parameter: canonical_mats = False
parameter: standardize = True
parameter: update_w0 = True
parameter: w0_threshold = 0.0
parameter: update_V = True
parameter: update_V_method = "full"
parameter: B_init_method = "enet"
parameter: max_iter = 5000
parameter: tol = 1e-2
parameter: verbose = False
parameter: save_model = False
parameter: glmnet_pred = False
depends: sos_variable("regional_data")

meta_info = regional_data['meta_info']
input: regional_data["data"], group_by = lambda x: group_by_region(x, regional_data["data"]), group_with = "meta_info"
output: f'{cwd:a}/{step_name[:-2]}_fold_{fold}/{name}.{_meta_info[0]}.mrmash.rds'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stdout = f"{_output:n}.stdout", stderr = f"{_output:n}.stderr", container = container, entrypoint = entrypoint
    library(pecotmr)
    # Load regional association data
    fdat = load_regional_multivariate_data(genotype = ${_input[0]:anr},
                                          phenotype = c(${",".join(['"%s"' % x.absolute() for x in _input[1::2]])}),
                                          covariate = c(${",".join(['"%s"' % x.absolute() for x in _input[2::2]])}),
                                          region = "${_meta_info[0]}",
                                          conditions = c('${_meta_info[2]}'),
                                          maf_cutoff = ${maf},
                                          mac_cutoff = ${mac},
                                          xvar_cutoff = ${var_cutoff},
                                          imiss_cutoff = ${imiss},
                                          matrix_y_min_complete = ${n_nonmiss_Y},
                                          keep_indel = ${"TRUE" if indel else "FALSE"})
    res = mrmash_wrapper(fdat$X, fdat$residual_Y, # here, X and Y are both matrices. X does not have missing data, Y may have it. Y is residualized (covariates removed) X is not.
                            ${sumstats_file:ar}, 
                            ${prior_matrices:ar}),
                            ${prior_weights:ar},
                            ${prior_grid:ar},
                            ${sample_partition:r}, # if available, will perform CV; otherwise will not perform CV
                            fold = ${fold},
                            nthreads = ${nthreads},
                            canonical_mats = ${"T" if canonical_mats else "F"},
                            standardize = ${"T" if standardize else "F"},
                            update_w0 = ${"T" if update_w0 else "F"},
                            w0_threshold = ${w0_threshold},
                            update_V = ${"T" if update_V else "F"},
                            update_V_method = "${update_V_method}",
                            B_init_method = "${B_init_method}",
                            max_iter = ${max_iter},
                            tol = ${tol},
                            verbose = ${"T" if verbose else "F"},
                            save_model = ${"T" if save_model else "F"},
                            glmnet_pred = ${"T" if glmnet_pred else "F"}
                          )
    st = proc.time()
    res$analysis_time = proc.time() - st
    res$analysis_script = load_script()
    res$dropped_samples = fdat$dropped_sample
    res$sample_names = colnames(fdat$residual_Y)
    saveRDS(res, ${_output:ar}, compress='xz')

## Functional regression fSuSiE for trait

In [ ]:
[fsusie_1]
parameter: max_L = 30
# remove a variant if it has more than imiss missing individual level data
parameter: imiss = 0.1
# MAF cutoff
parameter: maf = 0.0
# MAC cutoff, on top of MAF cutoff
# Here I set default to mac = 10 rather than using an MAF cutoff
# I don't set it to 5 because I'm not so sure of performance of SuSiE on somewhat infrequent variants
# MAC = 10 would not be too infrequenty for xQTL data where sample size is about ~1,000 at most (as of 2022)
parameter: mac = 10
# prior can be either of ["mixture_normal", "mixture_normal_per_scale"]
parameter: prior  = "mixture_normal_per_scale"
parameter: max_SNP_EM = 1000
# Remove indels if indel = False
parameter: indel = True
depends: sos_variable("regional_data")

meta_info = regional_data['meta_info']
input: regional_data["data"], group_by = lambda x: group_by_region(x, regional_data["data"]), group_with = "meta_info"
output: f'{cwd:a}/{step_name[:-2]}/{name}.{_meta_info[0]}.fsusie_{prior}.rds'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stdout = f"{_output:n}.stdout", stderr = f"{_output:n}.stderr", container = container, entrypoint = entrypoint
    # Load regional association data
    fdat = load_regional_association_data(genotype = ${_input[0]:anr},
                                          phenotype = c(${",".join(['"%s"' % x.absolute() for x in _input[1::2]])}),
                                          covariate = c(${",".join(['"%s"' % x.absolute() for x in _input[2::2]])}),
                                          region = ${'"%s:%s-%s"' % (_meta_info[1], _meta_info[2], _meta_info[3])},
                                          maf_cutoff = ${maf},
                                          mac_cutoff = ${mac},
                                          imiss_cutoff = ${imiss},
                                          y_as_matrix = FALSE,
                                          keep_indel = ${"TRUE" if indel else "FALSE"})
    # Fine-mapping with fSuSiE
    library("susiF.alpha")
    fitted = list()
    for (r in 1:length(fdat$residual_Y)) {
        st = proc.time()  
        fitted[[r]] <- susiF(fdat$residual_X[[r]],
                             fdat$residual_Y[[r]],
                             pos=fdat$phenotype_coordiates[[r]], #FIXME: this needs to be edited and added dto load_regional_association_data
                             L=${max_L},
                             prior="${prior}",
                             max_SNP_EM=${max_SNP_EM})
        fitted[[r]]$analysis_time = proc.time() - st
        fitted[[r]]$cs_corr = get_cs_correlation(fitted[[r]], X=fdat$residual_X[[r]])
        fitted[[r]]$cs_snps = names(fitted[[r]]$X_column_scale_factors[unlist(fitted[[r]]$sets$cs)])
        fitted[[r]]$variable_name = names(fitted[[r]]$pip)
        fitted[[r]]$coef = coef.susie(fitted[[r]])
        fitted[[r]]$analysis_script = load_script()
        fitted[[r]]$analysis_name = fdat$traits[[r]]
        fitted[[r]]$dropped_samples = fdat$dropped_sample[[r]]
        fitted[[r]]$sample_names = colnames(fdat$residual_Y[[r]])
    }
    saveRDS(fitted, ${_output:ar}, compress='xz')

## Functional regression fSuSiE with other modality

In [ ]:
[mvfsusie_1]
parameter: max_L = 30
# remove a variant if it has more than imiss missing individual level data
parameter: imiss = 0.1
# MAF cutoff
parameter: maf = 0.0
# MAC cutoff, on top of MAF cutoff
# Here I set default to mac = 10 rather than using an MAF cutoff
# I don't set it to 5 because I'm not so sure of performance of SuSiE on somewhat infrequent variants
# MAC = 10 would not be too infrequenty for xQTL data where sample size is about ~1,000 at most (as of 2022)
parameter: mac = 10
# prior can be either of ["mixture_normal", "mixture_normal_per_scale"]
parameter: prior  = "mixture_normal_per_scale"
parameter: max_SNP_EM = 1000

depends: sos_variable("regional_data")

meta_info = regional_data['meta_info']
input: regional_data["data"], group_by = lambda x: group_by_region(x, regional_data["data"]), group_with = "meta_info"
output: f'{cwd:a}/{step_name[:-2]}/{name}.{_meta_info[0]}.mvfsusie_{prior}.rds'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stdout = f"{_output:n}.stdout", stderr = f"{_output:n}.stderr", container = container, entrypoint = entrypoint
    # Load regional association data
    fdat = load_regional_association_data(genotype = ${_input[0]:anr},
                                          phenotype = c(${",".join(['"%s"' % x.absolute() for x in _input[1::2]])}),
                                          covariate = c(${",".join(['"%s"' % x.absolute() for x in _input[2::2]])}),
                                          region = ${'"%s:%s-%s"' % (_meta_info[1], _meta_info[2], _meta_info[3])},
                                          maf_cutoff = ${maf},
                                          mac_cutoff = ${mac},
                                          imiss_cutoff = ${imiss},
                                          y_as_matrix = FALSE)
    # Fine-mapping with mvfSuSiE
    library("mvf.susie.alpha")
    Y = map(fdat$residual_Y, ~left_join(fdat$X[,1]%>%as.data.frame%>%rownames_to_column("rowname"), .x%>%t%>%as.data.frame%>%rownames_to_column("rowname") , by = "rowname")%>%select(-2)%>%column_to_rownames("rowname")%>%as.matrix )
    fitted <- multfsusie(Y_f = list(Y[[1]],Y[[3]]), 
                         Y_u = Reduce(cbind, Y[[2]]),
                         pos = list(pos1 =fdat$phenotype_coordiates[[1]], pos2 = fdat$phenotype_coordiates[[3]]),
                         X=X,
                         L=${max_L},
                         data.format="list_df")
    saveRDS(fitted, ${_output:ar})

In [ ]:
[*_2]
input: group_by = "all"
output: f'{cwd}/{name}.cis_workhorse_output.txt'
python: expand= "$[ ]", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container = container, entrypoint = entrypoint
    import pandas as pd
    pd.DataFrame({"output" : [$[_input:ar,]]}).to_csv("$[_output]",index = False ,header = False, sep = "\t")